In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
directory = "../data/products.csv"
df = pd.read_csv(directory,delimiter="|")

In [5]:
df.head()

,asin,category,description,title,brand,price,feature,imageURL,price_category,price_range
0,B000DZXJSC,Cell Phones,"The Samsung SCH-a950 is red, hot and loaded wi...",Samsung A950 (Verizon Wireless),Samsung,17.99,['Streaming TV and Music With Verizon VCAST Se...,['https://images-na.ssl-images-amazon.com/imag...,Budget,$0 - $200
1,B000FJ1XWA,Cell Phones,The Nokia 1100 phone lets you conduct business...,Unlocked Nokia 1100,Nokia,40.08,"['Unlocked Nokia 1100', 'Unlocked Nokia 1100',...",['https://images-na.ssl-images-amazon.com/imag...,Budget,$0 - $200
2,B000G2TLIO,Cell Phones,The sleek VX8300 is built for serious mobile f...,Verizon or PagePlus LG VX8300 Great Condition ...,LG,27.57,['This product has been carefully audited and ...,['https://images-na.ssl-images-amazon.com/imag...,Budget,$0 - $200
3,B000G7LZCO,Cell Phones,The Moto Razr V3 is expertly crafted to delive...,"Motorola RAZR V3 Unlocked Phone with Camera, V...",Motorola,105.99,['This unlocked cell phone is compatible with ...,['https://images-na.ssl-images-amazon.com/imag...,Budget,$0 - $200
4,B000MXF2IK,Cell Phones,GSM/EDGE coverage (900/1800/1900 MHz)) Dimens...,Nokia 6300 Unlocked Triband Camera Business Phone,Nokia,40.08,['Nokia 6300 - Silver black (Unlocked) Mobile ...,['https://images-na.ssl-images-amazon.com/imag...,Budget,$0 - $200


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   asin            1215 non-null   object 
 1   category        1215 non-null   object 
 2   description     1215 non-null   object 
 3   title           1215 non-null   object 
 4   brand           1215 non-null   object 
 5   price           1215 non-null   float64
 6   feature         1215 non-null   object 
 7   imageURL        1215 non-null   object 
 8   price_category  1215 non-null   object 
 9   price_range     1215 non-null   object 
dtypes: float64(1), object(9)
memory usage: 95.0+ KB


In [9]:
print(df.columns)

Index(['asin', 'category', 'description', 'title', 'brand', 'price', 'feature',
       'imageURL', 'price_category', 'price_range'],
      dtype='object')


In [10]:
df.describe()

,price
count,1215.000000
mean,98.224864
std,97.127700
min,4.800000
25%,38.995000
50%,61.130000
75%,119.985000
max,706.090000


In [14]:
# Total number of products
print("Total number of products: ",df["asin"].nunique())

Total number of products:  1215


In [15]:
# Total number of categories
print("Total number of categories: ",df["category"].nunique())

Total number of categories:  1


In [16]:
# total rows with null values
print(df.isnull().sum())

asin              0
category          0
description       0
title             0
brand             0
price             0
feature           0
imageURL          0
price_category    0
price_range       0
dtype: int64


In [17]:
#remove column "category" as it is not required
df = df.drop(columns=["category"])

In [18]:
# Divide into 4 equal files
df1 = df.iloc[:int(len(df)/4)]
df2 = df.iloc[int(len(df)/4):int(len(df)/2)]
df3 = df.iloc[int(len(df)/2):int(3*len(df)/4)]
df4 = df.iloc[int(3*len(df)/4):]



In [19]:
# Write to csv
df1.to_csv("../data/products1.csv",index=False,sep="|")
df2.to_csv("../data/products2.csv",index=False,sep="|")
df3.to_csv("../data/products3.csv",index=False,sep="|")
df4.to_csv("../data/products4.csv",index=False,sep="|")


In [20]:
df_test = df1.iloc[:100]
df_test.to_csv("../data/products_test.csv",index=False,sep="|")


In [27]:
from langchain_community.document_loaders import CSVLoader
import sys
import csv
csv.field_size_limit(sys.maxsize)
file = "../data/products_test.csv"


loader = CSVLoader(file,csv_args={"delimiter": "|"})
documents = loader.load_and_split()

In [29]:
documents[0]

Document(page_content="asin: B000DZXJSC\ndescription: The Samsung SCH-a950 is red, hot and loaded with features, like V CAST Music. Add in Bluetooth&#xAE; (for certain profiles), a built-in 1.3 megapixel camera, camcorder and a stereo speakerphone, and you have one sizzling phone that's off-the-charts cool. Recommended for use with the Music Essential Kit.\ntitle: Samsung A950 (Verizon Wireless)\nbrand: Samsung\nprice: 17.99", metadata={'source': '../data/products_test.csv', 'row': 0})

In [30]:
from langchain_community.vectorstores.chroma import Chroma

def add_docs_to_existing_db(docs, embeddings,path = "../src/app/Chroma" ):
    # Load vector database
    db = Chroma(persist_directory=path, embedding_function=embeddings)
    if db is None:
        raise Exception("Vector database not found")
    db.add_documents(docs)
    db.persist()
    return db

In [33]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
add_docs_to_existing_db(documents, embeddings)